In [1]:
import sympy as sy

In [22]:
import re
def clean_latex(expr):
    s = sy.latex(expr)
    s = re.sub(r'\\operatorname{θ(\d)}{ *\\left \( *t *\\right *\)}', r'\\theta_\1', s)
    s = re.sub(r'\\operatorname{ϕ(\d)}{ *\\left \( *t *\\right *\)}', r'\\phi_\1', s)
    s = re.sub(r'([_^]){(\d+)}', r'\1\2', s)
    s = re.sub(r'\\frac{d}{d t} *(\\[a-zA-Z0-9_]+)', r'\\dot{\1}', s)
    s = re.sub(r'\\left *\( *([\\a-zA-Z0-9_{}]+) *\\right *\)', r'\1', s)
    s = s.replace(r'\left(m_{0} + m_{1} + m_{2}\right)', 'M_0')
    s = s.replace(r'\left(m_{1} + m_{2}\right)', 'M_1')
    s = s.replace(r'm_{2}', 'M_2')
    return s

In [26]:
N = 2
t = sy.symbols('t', real=True)
g = sy.symbols('g', positive=True)
l = sy.symbols(' '.join(f'l{i}' for i in range(N)), positive=True)
m = sy.symbols(' '.join(f'm{i}' for i in range(N)), positive=True)
θ = [theta(t) for theta in sy.symbols(' '.join(f'θ{i}' for i in range(N)), cls=sy.Function)]
dθ = [sy.diff(theta, t) for theta in θ]
ddθ = [sy.diff(theta, (t, 2)) for theta in θ]
ϕ =  [phi(t) for phi in sy.symbols(' '.join(f'ϕ{i}' for i in range(N)), cls=sy.Function)]
dϕ = [sy.diff(phi, t) for phi in ϕ]
ddϕ = [sy.diff(phi, (t, 2)) for phi in ϕ]

In [27]:
x = [l[0] * sy.sin(θ[0]) * sy.cos(ϕ[0])]
y = [l[0] * sy.sin(θ[0]) * sy.sin(ϕ[0])]
z = [-l[0] * sy.cos(θ[0])]
for i in range(1, N):
    x.append(x[i-1] + l[i] * sy.sin(θ[i]) * sy.cos(ϕ[i]))
    y.append(y[i-1] + l[i] * sy.sin(θ[i]) * sy.sin(ϕ[i]))
    z.append(z[i-1] - l[i] * sy.cos(θ[i]))

In [28]:
v2 = [sy.diff(xi, t)**2 + sy.diff(yi, t)**2 + sy.diff(zi, t)**2 for xi, yi, zi in zip(x, y, z)]

In [6]:
T = sum(vi2 * mi for vi2, mi in zip(v2, m)) / 2

In [7]:
U = sum(g * zi * mi for zi, mi in zip(z, m))

In [8]:
H = T + U
L = T - U

In [9]:
ddx = ddθ + ddϕ
dx = dθ + dϕ
x = θ + ϕ

In [10]:
eq = []
for i in range(N * 2):
    eq.append(sy.diff(L, dx[i], t) - sy.diff(L, x[i]))

In [11]:
mat1 = []
for i in range(N * 2):
    row = []
    mat1.append(row)
    for j in range(N * 2):
        row.append(sy.diff(eq[i], ddx[j]).simplify())

In [12]:
mat2 = []
for i in range(N * 2):
    row = []
    mat2.append(row)
    for j in range(N * 2):
        row.append(sy.diff(eq[i], (dx[j], 2)).simplify()/2)

In [13]:
left = sy.Matrix(eq) - sy.Matrix(mat1) * sy.Matrix(ddx) - sy.Matrix(mat2) * sy.Matrix([i * i for i in dx])

In [14]:
mat3 = []
for i in range(N * 2):
    row = []
    mat3.append(row)
    for j in range(N):
        row.append(sy.diff(left[i], dθ[j], dϕ[j]).simplify())

In [15]:
left2 = left - sy.Matrix(mat3) * sy.Matrix([a * b for a, b in zip(dθ, dϕ)])

In [16]:
for i in range(N * 2):
    print(left2[i].simplify())

g*l0*(m0 + m1)*sin(θ0(t))
g*l1*m1*sin(θ1(t))
0
0
